In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from prediction_util import *
from keras.preprocessing import sequence

C:\Users\robin\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\Users\robin\anaconda3\envs\crispr\lib\site-packages\keras\models.py:318: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [2]:
with open(r'./data/esp_seq_data_array.pkl', 'rb') as handle:
    esp_data = pickle.load(handle)
    
X,X_biofeat,y = esp_data

print("ESP data shape = ", X.shape)
print(X[:5])

print("Biofeat shape = ", X_biofeat.shape)
print(X_biofeat[:5])

print("y shape = ", y.shape)
print(y[:5])

ESP data shape =  (58617, 22)
[[1 2 2 2 2 2 2 2 2 2 4 3 4 4 2 2 2 2 4 4 4 3]
 [1 2 2 2 2 2 2 4 2 2 4 2 2 5 2 2 5 4 2 4 2 2]
 [1 2 2 2 2 2 2 4 2 4 2 2 5 4 2 2 5 2 4 4 5 3]
 [1 2 2 2 2 2 2 4 2 5 2 3 5 4 4 2 4 4 3 5 3 5]
 [1 2 2 2 2 2 2 4 4 4 5 3 2 5 2 3 2 5 4 4 3 4]]
Biofeat shape =  (58617, 11)
[[  0.           0.         -14.7        -13.3          0.
    1.           6.          46.86179822 -17.51643717   9.44740125
  -77.98477365]
 [  0.           0.         -14.2        -10.5          0.
    1.           6.          47.75596872 -10.18025086  10.85119385
  -77.98477365]
 [  0.           0.         -17.         -13.3          0.
    1.           8.          51.62485048  -6.43267221  16.55223598
  -77.98477365]
 [  0.          -1.7        -20.8        -17.1          0.
    1.           8.          51.73479019 -10.28003693  15.10601393
  -77.98477365]
 [  1.           0.         -21.4        -16.4          0.
    1.           8.          49.86316327  -9.83416842  17.33367012
  -77.98477

In [3]:
gRNA = ["GGCCGCCTCAGCCCACCAGCC" ,"GCCGCCTCAGCCCACCAGCCG", "GCTGTCCGCCGCCCGCCCCCA", "CTGTCCGCCGCCCGCCCCCAG", "CGCGTCCCGTCCCACCGTCGC"]

gRNA_ENC = make_data(gRNA)
print("gRNA_ENC shape = ", gRNA_ENC.shape)



for dnastring in gRNA:
    print("len = ", len(dnastring))

#C=4 G=5 A=2 T=3 START=1

gRNA_ENC shape =  (5, 22)
len =  21
len =  21
len =  21
len =  21
len =  21


In [4]:
#reconstruct gRNA from encoded dataset
def remake(X):
    remake = []
    for i in range(len(X)):
        dna = ""
        for j in range(len(X[i])):
            #if X[i][j] == 1:
                #dna+="START"
            if X[i][j] == 2:
                dna+="A"
            elif X[i][j] == 3:
                dna+="T"
            elif X[i][j] == 4:
                dna+="C"
            elif X[i][j] == 5:
                dna+="G"
        remake.append(dna)
    return remake

In [5]:
#remake function check
print(gRNA_ENC)
print(gRNA)
print(remake(gRNA_ENC))

[[1 5 5 4 4 5 4 4 3 4 2 5 4 4 4 2 4 4 2 5 4 4]
 [1 5 4 4 5 4 4 3 4 2 5 4 4 4 2 4 4 2 5 4 4 5]
 [1 5 4 3 5 3 4 4 5 4 4 5 4 4 4 5 4 4 4 4 4 2]
 [1 4 3 5 3 4 4 5 4 4 5 4 4 4 5 4 4 4 4 4 2 5]
 [1 4 5 4 5 3 4 4 4 5 3 4 4 4 2 4 4 5 3 4 5 4]]
['GGCCGCCTCAGCCCACCAGCC', 'GCCGCCTCAGCCCACCAGCCG', 'GCTGTCCGCCGCCCGCCCCCA', 'CTGTCCGCCGCCCGCCCCCAG', 'CGCGTCCCGTCCCACCGTCGC']
['GGCCGCCTCAGCCCACCAGCC', 'GCCGCCTCAGCCCACCAGCCG', 'GCTGTCCGCCGCCCGCCCCCA', 'CTGTCCGCCGCCCGCCCCCAG', 'CGCGTCCCGTCCCACCGTCGC']


Check if data actually turns out the same after decoding

In [6]:
seq = '''GCGGTGCCGCCCGCCGTGGCCGCCTCAGCCCACCAGCCGGGACCGCGAGCCATGCTGTCCGCCGCCCGCCCCCAGGGTTGTTAAAGCCAGACTGCGAACTCTCGCCACTGCCGCCACCGCCGCGTCCCGTCCCACCGTCGCGGGCAACAACCAAAGTCGCCGCAACTGCAGCACAGAGCGGGCAAAGCCAGGCAGGCCATGGGGCTCTGGGCGCTGTTGCCTGGCTGGGTTTCTGCTACGCTGCTGCTGGCGCTGGCCGCTCTGCCCGCAGCCCTGGCTGCCAACAGCAGTGGCCGATGGTGGGGTATTGTGAACGTAGCCTCCTCCACGAACCTGCTTACAGACTCCAAGAGTCTGCAACTGGTACTCGAGCCCAGTCTGCAGCTGTTGAGCCGCAAACAGCGGCGTCTGATACGCCAAAATCCGGGGATCCTGCACAGCGTGAGTGGGGGGCTGCAGAGTGCCGTGCGCGAGTGCAAGTGGCAGTTCCGGAATCGCCGCTGGAACTGTCCCACTGCTCCAGGGCCCCACCTCTTCGGCAAGATCGTCAACCGAGGCTGTCGAGAAACGGCGTTTATCTTCGCTATCACCTCCGCCGGGGTCACCCATTCGGTGGCGCGCTCCTGCTCAGAAGGTTCCATCGAATCCTGCACGTGTGACTACCGGCGGCGCGGCCCCGGGGGCCCCGACTGGCACTGGGGGGGCTGCAGCGACAACATTGACTTCGGCCGCCTCTTCGGCCGGGAGTTCGTGGACTCCGGGGAGAAGGGGCGGGACCTGCGCTTCCTCATGAACCTTCACAACAACGAGGCAGGCCGTACGACCGTATTCTCCGAGATGCGCCAGGAGTGCAAGTGCCACGGGATGTCCGGCTCATGCACGGTGCGCACGTGCTGGATGCGGCTGCCCACGCTGCGCGCCGTGGGCGATGTGCTGCGCGACCGCTTCGACGGCGCCTCGCGCGTCCTGTACGGCAACCGCGGCAGCAACCGCGCTTCGCGGGCGGAGCTGCTGCGCCTGGAGCCGGAAGACCCGGCCCACAAACCGCCCTCCCCCCACGACCTCGTCTACTTCGAGAAATCGCCCAACTTCTGCACGTACAGCGGACGCCTGGGCACAGCAGGCACGGCAGGGCGCGCCTGTAACAGCTCGTCGCCCGCGCTGGACGGCTGCGAGCTGCTCTGCTGCGGCAGGGGCCACCGCACGCGCACGCAGCGCGTCACCGAGCGCTGCAACTGCACCTTCCACTGGTGCTGCCACGTCAGCTGCCGCAACTGCACGCACACGCGCGTACTGCACGAGTGTCTGTGAGGCGCTGCGCGGACTCGCCCCCAGGAACGCTCTCCTCGAGCCCTCCCCCAAACAGACTCGCTAGCACTCAAGACCCGGTTATTCGCCCACCCGAGTACCTCCAGTCACACTCCCCGCGGTTCATACGCATCCCATCTCTCCCACTTCCTCCTACCTGGGGACTCCTCAAACCACTTGCCTGGGGCGGCATGAACCCTCTTGCCATCCTGATGGACCTGCCCCGGACCTACCTCCCTCCCTCTCCGCGGGAGACCCCTTGTTGCACTGCCCCCTGCTTGGCCAGGAGGTGAGAGAAGGATGGGTCCCCTCCGCCATGGGGTCGGCTCCTGATGGTGTCATTCTGCCTGCTCCATCGCGCCAGCGACCTCTCTGCCTCTCTTCTTCCCCTTTGTCCTGCGTTTTCTCCGGGTCCTCCTAAGTCCCTTCCTATTCTCCTGCCATGGGTGCAGACCCTGAACCCACACCTGGGCATCAGGGCCTTTCTCCTCCCCACCTGTAGCTGAAGCAGGAGGTTACAGGGCAAAAGGGCAGCTGTGATGATGTGGAAATGAGGTTGGGGGAACCAGCAGAAATGCCCCCATTCTCCCAGTCTCTGTCGTGGAGCCATTGAACAGCTGTGAGCCATGCCTCCCTGGGCCACCTCCTACCCCTTCCTGTCCTGCCTCCTCATCAGTGTGTAAATAATTTGCACTGAAACGTGGATACAGAGCCACGAGTTTGGATGTTGTAAATAAAACTATTTATTGTGCTGGGTCCCAGCCTGGTTTGCAAAGACCACCTCCAACCCAACCCAATCCCTCTCCACTCTTCTCTCCTTTCTCCCTGCAGCCTTTTCTGGTCCCTCTTCTCTCCTCAGTTTCTCAAAGATGCGTTTGCCTCCTGGAATCAGTATTTCCTTCCACTGTAGCTATTAGCGGCTCCTCGCCCCCACCAGTGTAGCATCTTCCTCTGCAGAATAAAATCTCTATTTTTA'''
enzyme = 'esp'

encoded = encode_only(seq, enzyme)

Constructing sencondary structure features.
N-None,N2-441
N-None,N2-441
Constructing nucleotide features.
N-None,N2-441
N-None,N2-441


In [7]:
print(encoded.shape)

decoded  = remake(encoded)

is_valid = True
for i in range(len(decoded)):
    if(len(decoded[i]) != 21):
        is_valid = False
        print("ERROR")
        break
print("Valid decode")


(441, 22)
Valid decode


Creating new dataset not decoded

In [8]:
def create_dataset(name, with_biofeat=False):
    filename = f'./data/{name}_seq_data_array.pkl'

    with open(filename, 'rb') as handle:
        data = pickle.load(handle)

    X,X_biofeat,y = data

    df = pd.DataFrame(columns=['seq' ,'y'])

    df['seq'] = remake(X)

    if with_biofeat:
        #make 11 biofeat columns
        for i in range(11):
            df['biofeat_'+ str(i)] = X_biofeat[:,i]

    df['y'] = y

    # print(df.head())
    # print(df.shape)

    #split df into train and test
    train, test = train_test_split(df, test_size=0.2, random_state=42)

    print(f"New  {name} dataset")
    print(train.head())
    print("Train shape = ", train.shape)
    print("Test shape = ", test.shape)


    if(with_biofeat):
        name += '_biofeat'

    #create dir if not exist
    if not os.path.exists(f'./data/{name}'):
        os.makedirs(f'./data/{name}')
    
    #remove files if exist
    if os.path.exists(f'./data/{name}/test_{name}_decoded.csv'):
        os.remove(f'./data/{name}/test_{name}_decoded.csv')
    if os.path.exists(f'./data/{name}/train_{name}_decoded.csv'):
        os.remove(f'./data/{name}/train_{name}_decoded.csv')
    

    #save to csv
    test.to_csv(f'./data/{name}/test_{name}_decoded.csv', index=False)
    train.to_csv(f'./data/{name}/train_{name}_decoded.csv', index=False)
    print("Saved to csv")
    print("")

In [9]:
#load data
create_dataset('esp', with_biofeat=True)
create_dataset('hf', with_biofeat=True)
create_dataset('wt', with_biofeat=True)

create_dataset('esp', with_biofeat=False)
create_dataset('hf', with_biofeat=False)
create_dataset('wt', with_biofeat=False)

New  esp dataset
                         seq        y  biofeat_0  biofeat_1  biofeat_2  \
39356  GCCTGCGAAAAGAGCACGCGC  0.32730        0.0       -5.7      -26.5   
32257  GAGGGGGCCGAGTCGATACTC  0.18817        0.0       -0.2      -28.4   
54966  GTCTTTGTATCACATCGCAGT  0.27452        0.0        0.0      -23.3   
8225   ACATCCTCCAGGTTTGCCAAA  0.18480        0.0       -0.3      -27.7   
47188  GGCCCATTAGTCGAAGGATGA  0.27203        0.0       -1.3      -25.6   

       biofeat_3  biofeat_4  biofeat_5  biofeat_6  biofeat_7  biofeat_8  \
39356      -12.3        1.0        0.0       13.0   62.32084    8.82756   
32257      -14.9        1.0        0.0       13.0   57.92372  -26.65553   
54966      -13.1        0.0        1.0        9.0   50.65496   -3.01763   
8225       -16.8        0.0        0.0       10.0   54.13293  -11.57110   
47188      -11.5        1.0        0.0       11.0   53.69039  -18.02492   

       biofeat_9  biofeat_10  
39356   16.80335   -48.92131  
32257   27.87738   -53.11

In [10]:
#compare different biofeats

# with open(r'./data/esp_seq_data_array.pkl', 'rb') as handle:
#     esp_data = pickle.load(handle)
# with open(r'./data/hf_seq_data_array.pkl', 'rb') as handle:
#     hf_data = pickle.load(handle)
# with open(r'./data/wt_seq_data_array.pkl', 'rb') as handle:
#     wt_data = pickle.load(handle)

# X_wt,X_biofeat_wt,y_wt = wt_data
# X_esp,X_biofeat_esp,y_esp = esp_data
# X_hf,X_biofeat_hf,y_hf = hf_data

# print("wt to esp")
# for i in range(10):
#     if(X_biofeat_wt[1,i] !=  X_biofeat_esp[1,i]):
#         print("Biofeat not equal")
#     if(X_wt[1,i] !=  X_esp[1,i]):
#         print("X not equal")
#     if(y_wt[1] !=  y_esp[1]):
#         print("y not equal")

# print("wt to hf")
# for i in range(10):
#     if(X_biofeat_wt[1,i] !=  X_biofeat_hf[1,i]):
#         print("Biofeat not equal")
#     if(X_wt[1,i] !=  X_hf[1,i]):
#         print("X not equal")
#     if(y_wt[1] !=  y_hf[1]):
#         print("y not equal")

# print("hf to esp")
# for i in range(10):
#     if(X_biofeat_hf[1,i] !=  X_biofeat_esp[1,i]):
#         print("Biofeat not equal")
#     if(X_hf[1,i] !=  X_esp[1,i]):
#         print("X not equal")
#     if(y_hf[1] !=  y_esp[1]):
#         print("y not equal")

We can conclude that only the enzyme is different so onl;y different y on datasets